In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE224705'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Defining molecular and cellular landscape behind non-response to Mycophenolate Mofetil in Systemic Lupus Erythematosus"
!Series_summary	"Lupus nephritis (LN) is one of the more severe systemic lupus erythematosus manifestations with the potential of developing into end stage kidney disease. Mycophenolate mofetil (MMF) is widely used for both induction and maintenance therapy for LN, but the one year complete renal response ranges from 30-40% in most trials. Reasons for non-response are still unknown.  Thus, anticipating lack of drug efficacy in a patient could lead to early introduction of advanced therapies.  A longitudinal cohort comprising gene-expression and clinical data of 97 MMF responder and 28 non-responder samples was retrospectively analyzed. Response to MMF was defined over time according to whether the protein/creatinine ratio in urine was reduced and maintained below 500 mg/g with the use of the treatment. Differential gene expressio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Gene Expression Data Availability
background_information = "Defining molecular and cellular landscape behind non-response to Mycophenolate Mofetil in Systemic Lupus Erythematosus"
if "gene-expression" in background_information or "gene expression" in background_information:
    is_gene_available = True

# Variable Availability and Data Type Conversion
data_dict = {0: ['days_from_first_visit: 0', 'days_from_first_visit: 52', 'days_from_first_visit: 82', 'days_from_first_visit: 124', 'days_from_first_visit: 143', 'days_from_first_visit: 35', 'days_from_first_visit: 126', 'days_from_first_visit: 219', 'days_from_first_visit: 295', 'days_from_first_visit: 384', 'days_from_first_visit: 629', 'days_from_first_visit: 91', 'days_from_first_visit: 85', 'days_from_first_visit: 68', 'days_from_first_visit: 109', 'days_from_first_visit: 38', 'days_from_first_visit: 157', 'days_from_first_visit: 96', 'days_from_first_visit: 97', 'days_from_first_visit: 268', 'days_from_first_visit: 79', 'days_from_first_visit: 170', 'days_from_first_visit: 435', 'days_from_first_visit: 560', 'days_from_first_visit: 163', 'days_from_first_visit: 27', 'days_from_first_visit: 58', 'days_from_first_visit: 284', 'days_from_first_visit: 382', 'days_from_first_visit: 478'],
             1: ['Sex: Female', 'Sex: Male'],
             2: ['age: 18.71', 'age: 17.66', 'age: 25.41', 'age: 25.77', 'age: 15.56', 'age: 19.38', 'age: 24.11', 'age: 22.25', 'age: 45.79', 'age: 22.26', 'age: 41.59', 'age: 19.84', 'age: 28.45', 'age: 36.34', 'age: 12.79', 'age: 23.12', 'age: 27.27', 'age: 16.92', 'age: 16.61', 'age: 17.02', 'age: 27.85', 'age: 32.14', 'age: 37.9', 'age: 24.83', 'age: 32.36', 'age: 22.85', 'age: 13.58', 'age: 60.05', 'age: 16.44', 'age: 36.28'],
             3: ['mmf_response: NO', 'mmf_response: YES'],
             4: ['sledai: 4', 'sledai: 0', 'sledai: 8', 'sledai: 10', 'sledai: 6', 'sledai: 12', 'sledai: 11', 'sledai: 2', 'sledai: 1', 'sledai: 3']}

# Chronic_kidney_disease (trait)
if 3 in data_dict and len(set(data_dict[3])) > 1:
    trait_row = 3
    def convert_trait(value):
        if "YES" in value:
            return 1
        elif "NO" in value:
            return 0
        return None

# Age
if 2 in data_dict and len(set(data_dict[2])) > 1:
    age_row = 2
    def convert_age(value):
        try:
            return float(value.split(": ")[1])
        except ValueError:
            return None

# Gender
if 1 in data_dict and len(set(data_dict[1])) > 1:
    gender_row = 1
    def convert_gender(value):
        if "Female" in value:
            return 0
        elif "Male" in value:
            return 1
        return None

# Save Metadata
save_cohort_info('GSE224705', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE224705.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7029843': [0.0, 18.71, 0.0], 'GSM7029844': [0.0, 18.71, 0.0], 'GSM7029845': [0.0, 18.71, 0.0], 'GSM7029846': [0.0, 18.71, 0.0], 'GSM7029847': [0.0, 18.71, 0.0], 'GSM7029848': [0.0, 17.66, 0.0], 'GSM7029849': [0.0, 17.66, 0.0], 'GSM7029850': [1.0, 25.41, 0.0], 'GSM7029851': [1.0, 25.41, 0.0], 'GSM7029852': [1.0, 25.41, 0.0], 'GSM7029853': [1.0, 25.41, 0.0], 'GSM7029854': [1.0, 25.41, 0.0], 'GSM7029855': [1.0, 25.41, 0.0], 'GSM7029856': [1.0, 25.41, 0.0], 'GSM7029857': [0.0, 25.77, 0.0], 'GSM7029858': [0.0, 25.77, 0.0], 'GSM7029859': [0.0, 15.56, 0.0], 'GSM7029860': [0.0, 15.56, 0.0], 'GSM7029861': [0.0, 19.38, 0.0], 'GSM7029862': [0.0, 19.38, 0.0], 'GSM7029863': [0.0, 24.11, 0.0], 'GSM7029864': [0.0, 24.11, 0.0], 'GSM7029865': [0.0, 24.11, 0.0], 'GSM7029866': [1.0, 22.25, 0.0], 'GSM7029867': [1.0, 22.25, 0.0], 'GSM7029868': [1.0, 22.25, 0.0], 'GSM7029869': [0.0, 45.79, 1.0], 'GSM7029870': [0.0, 45.79, 1.0], 'GSM7029871': [0.0, 22.26, 0.0], 'GSM7029872': [0.0, 22.26, 0.0], 'GSM70298